# 全局设置

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

In [ ]:
from hydra import compose, initialize
import os

# 加载项目层面的配置
with initialize(version_base=None, config_path="../config"):
    cfg = compose(config_name="config")
os.chdir(cfg.root)

## 加载数据

In [ ]:
from regimes_yrb.tools.statistic import (
    ratio_contribution,
    plot_pittitt_change_points,
    plot_ratio_contribution,
    pettitt_change_points,
)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec


plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "out"

COLORS = cfg.style.colors
period_colors = COLORS.period
region_colors = COLORS.region
index_colors = COLORS.index

index_colormap = matplotlib.colors.ListedColormap(index_colors, "indexed")
total_water_use_color = COLORS.total_WU

In [ ]:
# 加载阈值为 0.05的数据，即与黄河流域相交面积大于全市总面积 5% 的所有市
city_yr = pd.read_csv(cfg.db.perfectures)

In [ ]:
city_yr.head()

# 非直接用水比例

## 方法介绍

间接用水比例（Indirect Proportion of Water Use）来指示供水优先性，用于描述被消耗的水资源中，以间接形式惠益于人类的部分。

计算方法：
$$ I_{P} = \frac{WU_{non-pro}}{WU_{pro} + WU_{non-pro}} $$

其中i是年份，indirect和direct分别是为人类间接带来惠益的取用水方式和带来直接惠益取用水方式

indirect包括： 
- 工业用水量（IND），每年用于工业目的的取水量，包括自给自足的工业和与公共供水网络连接的工业。
- 城市服务业用水量（Urban service WU），指公共供水网络每年用于服务活动的直接用水量。

direct包括：
- 农业灌溉用水总量（IRR）：每年抽取的灌溉用水量，包括运输和田间施用期间的损失，但不包括牧场或水产养殖用水
- 城市居民用水（Urban domestic WU）：公共供水网络每年用于城镇居民直接取水的水量
- 农村居民用水（Rural domestic WU）：公共供水网络每年用于农村居民和牲畜直接使用的取水量
- 农村牲畜用水（Rural livestock WU）：公共供水网络用于牲畜饮水和清洁的年度取水量

In [ ]:
from regimes_yrb.priority import direct_ratio

# 产生直接惠益的用水方式
direct = [
    "IRR",
    "Urban domestic WU",
    "Rural domestic WU",
    "Rural livestock WU",
]

# 产生间接惠益的用水方式
indirect = ["IND", "Urban service WU"]

# 数据是用水量，每年加总
data = city_yr.groupby("Year").sum(numeric_only=True)

# 计算比例
priority, priority_contributions = direct_ratio(
    data, indirect=indirect, direct=direct
)

In [ ]:
priority_contributions

In [ ]:
from regimes_yrb.plot import plot_data

plot_data(
    priority,
    priority_contributions,
    [1977, 1993],
    "Indirect Proportion of Water Use, configuration",
    "Changes of configuration",
    "Total",
)

# 配水指数

## 方法介绍

模仿熵的公式，我们设计了一个衡量分配平均度的指标“配水指数(Water Distribution Index, WDI)”。我们假设最平均的水分配情况是，为每个行政区域某个部门分配到的用水量，都等于全流域该部门的平均用水比例。但实际上区域有别，有的区域会更集中发展某个用水部门，这就导致了水的分配不够平均，使得分配熵增大。而实际的分配情况与假设的平均分配情况，即为配水指数WDI，**WDI越大代表分配越平均，越小代表分配越不均匀**

![entropy](https://gitee.com/SongshGeo/PicGo_picbed/raw/master/img/diagram_water_distribution_index.jpg)

例如在上图中，左边展示的是假设的区域间均匀分配的情况。后者则是实际中分工各有差异，发展各有先后的情况。

我们使用 Configuration Entropy Metric $CEM$ 来表达这种配置上的均匀性

$$ CEM = \sum_{i=1}^n -log(p_{i}) * p_{i} $$

其中$i$ 代表配置水资源的单位（区域 or 部门） $p_i$ 代表了其中一个单位$i$分到水的比例。
对不同区域来说，当水资源完全平均分配，即对任意区域$i$，都有$p_i = 1/n $时，$Entropy$取最大值：

$$ CEM_{max} = \sum_{i=1}^n -log(p_{i}) * p_{i} = - log(\frac{1}{n}) $$

可见，$CEM$ 指标度量的是水资源配置“不均匀”的程度，类似于信息熵是对“混乱程度”的度量，即参与分配的各个单位之间，分配比例差距越大，则熵越小。
而我们的指标应反映随着社会发展，水资源配置在区域之间更加均衡（比例差距小）、整体满足不同用水部门的发展需求（比例差距减小）、但不同区域存在部门分工（比例差距增大）的趋势。

因此，用$CEM_r$代表流域各区域间的CEM，$CEM_s$代表流域各部门间的CEM, $CEM_{rs}$代表区域各部门间的CEM（详见附录）我们获得了表征流域用水配置的指标$I_C$：

$$ I_C = \frac{CEM_{r}*CEM_{s}}{CEM_{rs}}$$


### 配置熵指标：补充材料部分
衡量水资源配置的指标$I_C$由三部分水资源配置熵指标（CEM）构成：

$$ I_C = \frac{CEM_{r}*CEM_{s}}{CEM_{rs}}$$

(1) 描述区域间水资源配置的 $CEM_r$：

$$ CEM_r = \sum_{i=1}^4 -log(p_{i}) * p_{i} $$

其中$p_i$是区域$i$（$i=1$ to $4$ 分别代表 SR, UR, MR, or, DR）的水资源使用量（$WU_i$）在所有区域水资源消耗总量中所占的比例，即:

$$ p_i = \frac{WU_i}{\sum_{i=1}^4 WU_i} $$

(2) 描述部门间水资源配置的 $CEM_s$:

$$ CEM_s = \sum_{j=1}^3 -log(p_{j}) * p_{j} $$

其中$p_j$是用水部门$j$（$j=1$ to $3$ 分别代表 agriculture, industry, domestic and services）的水资源使用量（$WU_j$）在所有部门水资源消耗总量中所占的比例，即:

$$ p_j = \frac{WU_j}{\sum_{j=1}^3 WU_j} $$


(3) 描述区域各部门间水资源配置分工的 $CEM_{rs}$:

$$ CEM_{rs} = \frac{1}{3} * \sum_{j=1}^3 \sum_{i=1}^4 -log(p_{ij} * p_{ij})$$

其中$p_{ij}$是区域$i$在部门$j$上的用水$WU_{ij}$占所有区域在部门$j$上总用水的比例，即：

$$ p_{ij} = \frac{WU_ij}{\sum_{i=1}^4 WU_ij} $$

In [ ]:
from regimes_yrb.allocation import entropy_and_contribution

allocation, entropy_contributions = entropy_and_contribution(city_yr)
plot_data(
    allocation,
    entropy_contributions,
    [1977, 1993],
    "Water Distribution Index, WDI",
    "Changes of WDI",
    "Divisions",
)

# SFV结构性缺水指数

## 方法介绍

人类利用科技手段不断的管理水资源，因此简单的物理缺水指数无法合理评估结构性水资源短缺在社会-水系统演化与过渡中所起的作用。因此，我们参考 Qin et al., 2019中提出的结构性缺水指数来评价流域缺水。该指数考虑了修建水库等管理措施，以及用水结构变化对水资源短缺情况评价的影响。

利用该方法指示水资源压力$I_S$，认为全流域的水资源压力是各区域SFV指数的均值，即：

$$ I_S = \frac{1}{4} * \sum_{i=1}^4 SFV_{i} $$

而计算某区域$i$的SFV指数，需要考虑三个权重相等的方面，标准化后计算SFV指数：
$$ SFV_i = a * V_i + b $$
$$ V_i = \frac{A_{i, normalize} + B_{i, normalize} + C_{i, normalize}}{3} $$
$$ a = \frac{1}{V_{max} - V_{min}}; $$
$$ b = \frac{1}{V_{min} - V_{max}} * V_{min} $$

（注：原文献中使用的常数为100，得到的指数结果位于$0~100$之间，这里为了和其余两指数保持$0~1$的相同数量级，将该系数调整为1）
- 首先是总的耗水量占区域多年平均径流量的比例：
$$ A_{i, j} = \frac{WU_{i,j}}{R_{i, avg}} $$

其中平均径流量用如下公式计算：
$$ R_{i, avg} = \frac{1}{j} * \sum_{j=1}^j R_{i} $$

- 其次是非弹性用水量占多年平均径流量的比例：
$$ B_{i, j} = \frac{WU_{inflexible}}{R_{i, avg}} $$

对于非弹性用水，Qin在文章中给出了如下划分：
For inflexible consumption, we include (according to Qin et al., 2019):    
- (1) Freshwater consumed for irrigation of perennial crops.
- (2) Water  evaporated during cooling of thermal power plants. √
- (3) Water evaporated from reservoirs.
- (4) Basic water allotments for humans and livestock. √

由于数据限制，本研究中，取其中（2）、（4）的和为非弹性用水。

- 最后，还要考虑水库的库容能力及调蓄作用对应对天然径流波动的积极影响：
$$ C1_{i, j} = \frac{R_{i, std}}{R_{i, avg}} $$

$$ C2_{i} = \frac{RC_{i}}{R_{i, avg}}, \ if RC < R_{i, avg} $$

$$ C2_{i} = 1, \ if RC >= R_{i, avg} $$

$$ C_i = C1_i * (1 - C2_i) $$

其中$RC_i$是该区域的水库总库容，$R_{i, std}$是多年径流的标准差，衡量天然径流的波动。

Qin, Y. et al. Flexibility and intensity of global water use. Nature Sustainability 2, 515–523 (2019).

### 加载径流和水库数据

In [ ]:
# 加载径流和水库数据
use_cols = {
    "唐乃亥": "SR",  # 唐乃亥控制源区
    "头道拐": "UR",  # 头道拐控制上游
    "花园口": "MR",  # 花园口控制中游
    "利津": "DR",  # 利津控制下游
}
measured_runoff = pd.read_csv(cfg.db.runoff, index_col="年份")
measured_runoff = measured_runoff.loc[:, use_cols.keys()]
measured_runoff.rename(use_cols, axis=1, inplace=True)

In [ ]:
# 水库库容数据
reservoirs_capacity = pd.read_csv(cfg.db.reservoirs, index_col=0)

# 水库库容累积相加数据
reservoirs_capacity_cumulating = reservoirs_capacity.cumsum()

In [ ]:
consumptions = pd.read_csv(cfg.db.consumptions)

## 计算三部分指数

In [ ]:
## 计算三个部分的指数
from regimes_yrb.scarcity import (
    calculate_index_c,
    calculate_index_a,
    calculate_index_b,
)


index_a = calculate_index_a(runoff=measured_runoff, consumptions=consumptions)
index_b = calculate_index_b(
    runoff=measured_runoff,
    consumptions=consumptions,
    inflexible_wu=cfg.inflexible_wu,
)
index_c = calculate_index_c(
    runoff=measured_runoff, reservoirs=reservoirs_capacity_cumulating
)

In [ ]:
from regimes_yrb.scarcity import calc_sfv

sfv, sfv_contribution = calc_sfv(
    measured_runoff,
    consumptions,
    reservoirs_capacity_cumulating,
    cfg.inflexible_wu,
)

# 综合耦合指数

我们认为水资源利用制度，与水资源利用的三个维度紧密相关：
- 用水压力（stress, S）
- 用水优先性（configuration, P）
- 用水配置（priority, C）

根据经验，我们认为这三者与社会发展$(Dev.)$具有方向性关系:

- 社会的发展通常伴随着用水向社会经济系统倾斜，用水方式优先向收益更高的非供给性方式倾斜：
$$ Dev \propto P $$
- 社会发展通常伴随着更具结构性的水资源配置，如区域部门之间的分工合作，以及区域的统筹配置：
$$ Dev \propto C $$ 
- 可持续的社会发展应该通过技术手段有效缓解发展过程中产生的水资源压力，才能实现可持续发展：
$$ Dev \propto S^{-1} $$

将三者合一起，即：
$$ Dev. \propto P*C*S^{-1} $$

在上述假设的基础上，我们要构建流域综合耦合指数(Integrated Water Resources Utilization, IWRU)，使 IWRU 有效表征与用水相关的三个维度。首先为每个维度选择一个合适的指示因子（indicator, $I_x$, 其中$x=P, C or S$）。将上式进行自然对数转换，从而让三个维度之间变成加减关系：
$$ Dev. \propto ln(I_P) + ln(I_C) - ln(I_S) $$

然后为了给定 IWRU 确定的值域，同时让三个维度之间等权重，对三者进行标准化处理，得到我们的综合耦合指数 IWRU：
$$ IWRU = norm(ln(I_P)) + norm(ln(I_C)) - norm(ln(I_S)) $$

In [ ]:
from regimes_yrb.index import integrated_water_governance_index

iwgi = integrated_water_governance_index(
    priority=priority, scarcity=sfv, allocation=allocation
)
iwgi.loc[iwgi.index < 1986, "stage"] = "P1"
iwgi.loc[(1986 <= iwgi.index) & (iwgi.index < 2001), "stage"] = "P2"
iwgi.loc[2001 <= iwgi.index, "stage"] = "P3"

In [ ]:
import seaborn as sns

sns.pairplot(iwgi, hue="stage")

## Fig.3 三元图

对三元图来说，计算的是每个维度 $X$ 对 regime 的影响（Impact）

$$ Impact_X = \frac{I'_X}{\sum_X I'_X} $$

In [ ]:
from regimes_yrb.plot import plot_ternary

ax = plot_ternary(iwgi)
# plt.savefig("../figures/main/phases.jpg", dpi=300)
# plt.savefig("../figures/main/phases.pdf", dpi=300)
plt.show();

## 计算贡献度

首先是计算各个指标（$I_x, x = P, S, or C$）的贡献度：

$$ IWRU = I'_P + I'_C - I'_S $$

对每个起始年份为$y1$，截止年份为$y2$的时间段，有：

$$ \Delta IWRU = IWRU_{y2} - IWRU_{y1} $$

即：

$$ \Delta IWRU = (I'_{P_{y2}} + I'_{C_{y2}} - I'_{S_{y2}}) - (I'_{P_{y1}} + I'_{C_{y1}} - I'_{S_{y1}}) = \Delta I'_P + \Delta I'_C + (-\Delta I'_S) $$

则每个指标$x$ ($P$, $C$, or $S$)对IWRU变化量的贡献度$Contribution_x$为：

$$ Contribution_x = \frac{\Delta I'_x}{|\Delta IWRU|} $$

其影响是：

$$ Impact_X = \frac{\Delta I'_X}{\sum_X |\Delta I'_X|} $$

In [ ]:
from regimes_yrb.index import calc_contribution

changes_contribution = calc_contribution(iwgi, threshold=cfg.alpha_threshold)
changes_contribution

# 各个指数对 iwru 在每个阶段发生变化的贡献量
print("第三阶段 WDI 的贡献是较第二阶段增长的倍数：")
changes_contribution["A"].iloc[2] / changes_contribution["A"].iloc[1] - 1

print("第三阶段 configuration 的贡献是较第二阶段增长的倍数：")
changes_contribution["P"].iloc[2] / changes_contribution["P"].iloc[1] - 1

print("第三阶段 SFV 的贡献是较第二阶段增长的倍数：")
changes_contribution["S"].iloc[2] / changes_contribution["S"].iloc[1] - 1

In [ ]:
fig, ax = plt.subplots()

labels = ["P", "A", "S"]
ys = [iwgi[col] / 3 for col in labels]
# 使用【堆积图】绘制子区域
ax.stackplot(
    iwgi.index, *ys, colors=index_colors, alpha=0.4, zorder=0, labels=labels
)
# 使用【散点图】绘制黄河流域总体值
# ax.scatter(x, y_all, zorder=2, edgecolor="#39AEA9", color="#39AEA9", s=50, alpha=.4)
# 使用【直线图】展示拟合的直线
# ax.plot(x, y_linear_fit, ls="--", lw=2, zorder=3)
ax.axvline(1978, ls=":", color="gray")
ax.axvline(2001, ls=":", color="gray")
ax.legend()

In [ ]:
from regimes_yrb.tools.statistic import zscore, get_optimal_fit_linear
from scipy import stats

k_results = pd.DataFrame(index=["P1", "P2", "P3"])
# b_results = pd.DataFrame(index=["P1", "P2", "P3"])
corr_results = pd.DataFrame(index=["P1", "P2", "P3"])
p_results = pd.DataFrame(index=["P1", "P2", "P3"])
breakpoints = [1965, 1978, 2001, 2013]

_, axs = plt.subplots(1, 3, figsize=(10, 3))
for i, yr_start in enumerate(breakpoints[:3]):
    ax = axs[i]
    yr_end = breakpoints[i + 1]
    temp_data = iwgi.loc[yr_start:yr_end]
    for col in iwgi.iloc[:, :3]:
        x, y = temp_data.index, temp_data[col].values
        y_sim, k, _ = get_optimal_fit_linear(x, y)
        k_results.loc[f"P{i+1}", col] = k
        ax.scatter(zscore(temp_data[col]), zscore(y_sim), label=f"{col}")
        if col != "IWGI":
            corr, p_val = stats.spearmanr(
                temp_data[col].values, temp_data["IWGI"].values
            )
            corr_results.loc[f"P{i+1}", col] = corr
            p_results.loc[f"P{i+1}", col] = p_val
plt.show();

In [ ]:
from regimes_yrb.plot import plot_demonstrate


plot_demonstrate()

## 最终作图

In [ ]:
# 使用图片的比例来定位
def get_position_by_ratio(ax, x_ratio, y_ratio):
    """
    使用图片的比例来返回定位，从而更好的控制说明文字的位置
    ax: 一个 matplotlib 的画图轴对象
    x_ratio: 横坐标的比例位置
    y_ratio: 纵坐标的比例位置
    """
    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()
    x = (x_max - x_min) * x_ratio + x_min
    y = (y_max - y_min) * y_ratio + y_min
    return x, y

In [ ]:
# 绘图
fig = plt.figure(constrained_layout=False, figsize=(8, 4.4))
fig.subplots_adjust(left=0.10, right=0.95, top=0.95, bottom=0.10)

gs = GridSpec(
    5,
    2,
    figure=fig,
    width_ratios=[4, 5],
    height_ratios=[0.05, 1, 0.05, 1, 0.0],
)
ax1 = fig.add_subplot(gs[0:2, 0])
ax2 = fig.add_subplot(gs[3:5, 0])
ax3 = fig.add_subplot(gs[1:-1, 1])
ax4 = fig.add_subplot(355)
plot_demonstrate(ax4)

tax = plot_ternary(iwgi, ax=ax3)

# 使用【堆积图】绘制子区域
ax1.stackplot(
    iwgi.index,
    *ys,
    colors=index_colors,
    alpha=0.6,
    zorder=0,
    labels=labels,
    edgecolor="white",
)
wcci_slopes = plot_pittitt_change_points(
    iwgi["IWGI"],
    ax=ax1,
    colors=period_colors,
    p_shr=cfg.alpha_threshold,
    edgecolor="white",
    s=40,
)

# 绘制相关系数
labels = ["P", "A", "S"]
# corr_results.columns = ["scarcity", "priority", "allocation"]
corr_results[labels].plot.bar(
    ax=ax2, colormap=index_colormap, edgecolor="white", width=0.9
)

ys = [iwgi[col] / 3 for col in labels]


# 修饰图片1
# ax1.set_xlabel('Year')
ax1.set_ylabel("Index")
ax1.set_xlim(1965, 2014)
# ax1.set_ylim(0.04, 0.105)
# ax1.set_yticks(np.arange(0.04, 0.105, 0.02))
# ax1.text(1968, 0.4, 'A.', ha='center', va='center', weight='bold', size='large')
ax1.legend(loc="upper right", ncol=2)

# 修饰图片3
# ax3.set_xlabel("Different periods")
ax2.set_ylabel("Correlations")
for tick in ax3.get_xticklabels():  # 旋转角度
    tick.set_rotation(0)  # 轴标签旋转
ax2.axhline(y=0, c="gray", lw=1, ls="--")
# ax3.set_xticklabels(["1965-1977", "1978-2000", "2001-2013"])
ax2.set_yticks(np.arange(-0.5, 0.8, 0.3))
ax2.set_yticklabels(["-.5", "-.2", "0.1", "0.4", "0.7"])
ax2.tick_params(axis="x", tickdir="in", bottom=False, labelrotation=0)
# ax3.text(-0.2, 0.7, 'B.', ha='center', va='center', weight='bold', size='large')
ax2.axvline(1.5, ls=":", color="gray")
ax2.axvline(0.5, ls=":", color="gray")
ax2.set_xlim(-0.5, 2.5)

v_space = 0.01
for i, period in enumerate(corr_results.index):
    for j, indicator in enumerate(corr_results.columns):
        height = corr_results[labels].iloc[i, j] + v_space
        location = i + (-0.3, 0, 0.3)[j]
        if p_results[labels].iloc[i, j] < 0.05:
            ax2.text(
                location,
                height,
                "* *",
                horizontalalignment="center",
                weight="bold",
                color="black",
            )

# 调整坐标轴显示
for ax in [ax1, ax2, ax4]:
    ax.spines["top"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["left"].set_visible(True)
    ax.spines["right"].set_visible(False)
    ax.grid(True, axis="y", color="white", ls=":")

ax1.spines["bottom"].set_visible(True)
ax4.spines["left"].set_visible(False)

legend_handles = []
legend_labels = []
for handle, label in zip(*ax2.get_legend_handles_labels()):
    legend_handles.append(handle)
    legend_labels.append(label.title() + f" ({label[0].title()})")

for handle, label in zip(*ax3.get_legend_handles_labels()):
    legend_handles.append(handle)
    legend_labels.append(label)

for i, ax in enumerate([ax1, ax2, ax3]):
    ax.get_legend().remove()
    ratio_x, ratio_y = 0.95, 0.9
    if i == 2:
        ratio_x = 0.05
        ratio_y = 0.406
    x, y = get_position_by_ratio(ax, ratio_x, ratio_y)
    label = ["A.", "B.", "C."][i]
    ax.text(x, y, label, ha="center", va="center", weight="bold", size="large")

fig.legend(
    loc=(0.46, 0.60),
    ncol=1,
    handles=legend_handles,
    labels=legend_labels,
    handletextpad=1,
    handleheight=1,
    markerscale=1,
    frameon=False,
    title="Legend:",
    title_fontproperties={"weight": "bold", "size": 10},
)

# 储存和显示图片
# plt.savefig("../figures/main/index.pdf", dpi=300)
# plt.savefig("../figures/main/index.jpg", dpi=300)
plt.show();

In [ ]:
temp_df = iwgi.loc[2001:2013, ["P", "A", "S"]].mean()

temp_df / temp_df.sum()

In [ ]:
corr_results

## 对阈值变化的敏感性分析